In [1]:
!pip install gdown

Download data

In [2]:
#!gdown 1iSZwy5tkx9T0TZrB-0twEaf7jAadhXfS
!gdown 1741Os55KZDupJpi7m-mOxmtq-dWg_3IA

Downloading...
From (original): https://drive.google.com/uc?id=1741Os55KZDupJpi7m-mOxmtq-dWg_3IA
From (redirected): https://drive.google.com/uc?id=1741Os55KZDupJpi7m-mOxmtq-dWg_3IA&confirm=t&uuid=f417cd13-1a98-45f6-b97b-6a187a04cb55
To: /content/competitionData.tar.gz
100% 3.67G/3.67G [00:56<00:00, 64.4MB/s]


In [3]:
!tar -xzvf "competitionData.tar.gz"

./competitionData/
./competitionData/competitionHoldOut/
./competitionData/competitionHoldOut/t12.2022.05.24.mat
./competitionData/competitionHoldOut/t12.2022.06.14.mat
./competitionData/competitionHoldOut/t12.2022.06.07.mat
./competitionData/competitionHoldOut/t12.2022.07.27.mat
./competitionData/competitionHoldOut/t12.2022.08.02.mat
./competitionData/competitionHoldOut/t12.2022.06.28.mat
./competitionData/competitionHoldOut/t12.2022.08.11.mat
./competitionData/competitionHoldOut/t12.2022.06.21.mat
./competitionData/competitionHoldOut/t12.2022.07.14.mat
./competitionData/competitionHoldOut/t12.2022.06.16.mat
./competitionData/competitionHoldOut/t12.2022.07.21.mat
./competitionData/competitionHoldOut/t12.2022.08.13.mat
./competitionData/competitionHoldOut/t12.2022.05.26.mat
./competitionData/competitionHoldOut/t12.2022.07.05.mat
./competitionData/competitionHoldOut/t12.2022.06.02.mat
./competitionData/train/
./competitionData/train/t12.2022.05.24.mat
./competitionData/train/t12.2022.07

In [4]:
sessionNames = ['t12.2022.04.28',  't12.2022.05.26',  't12.2022.06.21', 't12.2022.07.21',  't12.2022.08.13',
't12.2022.05.05',  't12.2022.06.02',  't12.2022.06.23',  't12.2022.07.27',  't12.2022.08.18',
't12.2022.05.17',  't12.2022.06.07',  't12.2022.06.28',  't12.2022.07.29',  't12.2022.08.23',
't12.2022.05.19',  't12.2022.06.14',  't12.2022.07.05',  't12.2022.08.02',  't12.2022.08.25',
't12.2022.05.24',  't12.2022.06.16',  't12.2022.07.14',  't12.2022.08.11']
sessionNames.sort()

In [5]:
!pip install g2p_en

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 23.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for distance: filename=Distance-0.1.3-py3-none-any.whl size=16258 sha256=780c35e8faf239c878829ddf179b507ac9d7470a11317dd51de1d45e58c9ae60
  Stored in directory: /root/.cache/pip/wheels/e8/bb/de/f71bf63559ea9a921059a5405806f7ff6ed612a9231c4a9309
Successfully built distance


Data preparation From baseline algorithm

In [6]:
import pandas as pd

In [7]:
import re
from g2p_en import G2p
import numpy as np

g2p = G2p()
PHONE_DEF = [
    'AA', 'AE', 'AH', 'AO', 'AW',
    'AY', 'B',  'CH', 'D', 'DH',
    'EH', 'ER', 'EY', 'F', 'G',
    'HH', 'IH', 'IY', 'JH', 'K',
    'L', 'M', 'N', 'NG', 'OW',
    'OY', 'P', 'R', 'S', 'SH',
    'T', 'TH', 'UH', 'UW', 'V',
    'W', 'Y', 'Z', 'ZH'
]
PHONE_DEF_SIL = PHONE_DEF + ['SIL']

def phoneToId(p):
    return PHONE_DEF_SIL.index(p)

def IdToPhone(p):
    return PHONE_DEF_SIL.index(p)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [8]:
import scipy

def loadFeaturesAndNormalize(sessionPath):

    dat = scipy.io.loadmat(sessionPath)

    input_features = []
    transcriptions = []
    frame_lens = []
    block_means = []
    block_stds = []
    n_trials = dat['sentenceText'].shape[0]

    #collect area 6v tx1 and spikePow features
    for i in range(n_trials):

#         features = np.concatenate([dat['tx1'][0,i][:,0:128], dat['spikePow'][0,i][:,0:128]], axis=1)
        features = np.array(dat['tx1'][0,i][:,0:128]) * np.array(dat['spikePow'][0,i][:,0:128])


        sentence_len = features.shape[0]
        sentence = dat['sentenceText'][i].strip()

        input_features.append(features)
        transcriptions.append(sentence)
        frame_lens.append(sentence_len)

    #block-wise feature normalization
    blockNums = np.squeeze(dat['blockIdx'])
    blockList = np.unique(blockNums)
    blocks = []
    for b in range(len(blockList)):
        sentIdx = np.argwhere(blockNums==blockList[b])
        sentIdx = sentIdx[:,0].astype(np.int32)
        blocks.append(sentIdx)

    for b in range(len(blocks)):
        feats = np.concatenate(input_features[blocks[b][0]:(blocks[b][-1]+1)], axis=0)
        feats_mean = np.mean(feats, axis=0, keepdims=True)
        feats_std = np.std(feats, axis=0, keepdims=True)
        for i in blocks[b]:
            input_features[i] = (input_features[i] - feats_mean) / (feats_std + 1e-8)

    #convert to tfRecord file
    session_data = {
        'inputFeatures': input_features,
        'transcriptions': transcriptions,
        'frameLens': frame_lens
    }

    return session_data

In [9]:
import os

def getDataset(fileName):
    session_data = loadFeaturesAndNormalize(fileName)

    allDat = []
    trueSentences = []
    seqElements = []

    for x in range(len(session_data['inputFeatures'])):
        allDat.append(session_data['inputFeatures'][x])
        trueSentences.append(session_data['transcriptions'][x])

        thisTranscription = str(session_data['transcriptions'][x]).strip()
        thisTranscription = re.sub(r'[^a-zA-Z\- \']', '', thisTranscription)
        thisTranscription = thisTranscription.replace('--', '').lower()
        addInterWordSymbol = True

        phonemes = []
        for p in g2p(thisTranscription):
            if addInterWordSymbol and p==' ':
                phonemes.append('SIL')
            p = re.sub(r'[0-9]', '', p)  # Remove stress
            if re.match(r'[A-Z]+', p):  # Only keep phonemes
                phonemes.append(p)

        #add one SIL symbol at the end so there's one at the end of each word
        if addInterWordSymbol:
            phonemes.append('SIL')

        seqLen = len(phonemes)
        maxSeqLen = 128
        seqClassIDs = np.zeros([maxSeqLen]).astype(np.int32)
        seqClassIDs[0:seqLen] = [phoneToId(p) + 1 for p in phonemes]
        seqElements.append(seqClassIDs)

    newDataset = {}
    newDataset['sentenceDat'] = allDat
    newDataset['transcriptions'] = trueSentences
#     newDataset['phonemes'] = seqElements

    timeSeriesLens = []
#     phoneLens = []
    for x in range(len(newDataset['sentenceDat'])):
        timeSeriesLens.append(newDataset['sentenceDat'][x].shape[0])

    newDataset['timeSeriesLens'] = np.array(timeSeriesLens)

    return pd.DataFrame.from_dict(newDataset)

In [10]:
trainDatasets = pd.DataFrame()
testDatasets = pd.DataFrame()
competitionDatasets = []

dataDir = './competitionData'

for dayIdx in range(len(sessionNames)):
    print(dayIdx)
    trainDataset = getDataset(dataDir + '/train/' + sessionNames[dayIdx] + '.mat')
    testDataset = getDataset(dataDir + '/test/' + sessionNames[dayIdx] + '.mat')
    trainDatasets = pd.concat([trainDatasets,trainDataset],ignore_index=True)
    testDatasets = pd.concat([testDatasets,testDataset],ignore_index=True)

    if os.path.exists(dataDir + '/competitionHoldOut/' + sessionNames[dayIdx] + '.mat'):
        dataset = getDataset(dataDir + '/competitionHoldOut/' + sessionNames[dayIdx] + '.mat')
        competitionDatasets.append(dataset)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [11]:
dataDir = './competitionData'

competitionDays = []
for dayIdx in range(len(sessionNames)):
    if os.path.exists(dataDir + '/competitionHoldOut/' + sessionNames[dayIdx] + '.mat'):
        competitionDays.append(dayIdx)
print(competitionDays)

[4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18, 19, 20]


In [12]:
import pickle

allDatasets = {}
allDatasets['train'] = trainDatasets
allDatasets['test'] = testDatasets
allDatasets['competition'] = competitionDatasets

with open(dataDir+'/Datasets', 'wb') as handle:
    pickle.dump(allDatasets, handle)


In [13]:
import numpy as np
import random

## Data Preparation

continue


In [14]:
# !python -m spacy download en

In [15]:
import spacy
import random
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import math

# from torchtext.data import Field, BucketIterator

In [16]:
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)

INPUT_DIM = 1135#
OUTPUT_DIM = 127#
ENC_EMB_DIM = 128#
DEC_EMB_DIM = 128#
ENC_HID_DIM = 256#
DEC_HID_DIM = 256#
N_LAYERS = 1
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
BATCH_SIZE = 10 #20
SOS = 35 #(#)
EOS = 64 #(@)

In [17]:
def createBatch(df): #make the same length
    max_time = 1024
    df = np.append(df, [[0 for k in range(128)] for j in range(max_time - df.shape[0])], axis=0)
    return df
def create_sentences(sent):
    max_time = 128
    sent = np.append(sent, [0 for j in range(max_time - sent.shape[0])], axis=0)
    return sent


In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [19]:
punctuation_txt = [' ',',', '`', '-', '!', '?', '.', ':',';',"'"]

In [20]:
class BrainDataset(Dataset):
    def __init__(self, df):
        self.input_name = 'sentenceDat'
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df['transcriptions'][idx]
        brainDatRaw = self.df['sentenceDat'][idx]
        brainDatRaw = createBatch(brainDatRaw)

        target = [ord(i.lower()) for i in text if i not in punctuation_txt]
        target.insert(0,SOS)
        target.insert(-1,EOS)
        target=target[::-1] #sentence chars to didits
        target= create_sentences(np.array(target))

        return {"brain":brainDatRaw, "target": target}

In [21]:
trainDs = BrainDataset(trainDatasets)
testDS = BrainDataset(testDatasets)

In [22]:
train_dataloader = DataLoader(trainDs, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(testDS, batch_size=BATCH_SIZE)

## Model

In [23]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, n_layers, dropout):
        super().__init__()

        self.conv = nn.Conv2d(1, 1, (1,4))
        self.pool = nn.MaxPool2d(kernel_size=(1,4))

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.input_dim = input_dim
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, n_layers, dropout=dropout,
                          bidirectional=True)
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

    def forward(self, src_batch):
        x = self.pool(F.leaky_relu(self.conv(src_batch)))
        x= self.pool(F.leaky_relu(self.conv(x)))
        x = self.pool(F.leaky_relu(self.conv(x)))
        x= x.to(torch.int64)
        x= x.squeeze()
        x = torch.permute(x,(1,0))

        ## Add SOS = 35 (#) and EOS = 64 (@)
        sos = torch.Tensor([[35]*BATCH_SIZE]).to(device)
        eos = sos = torch.Tensor([[64]*BATCH_SIZE]).to(device)
        x = torch.cat((sos,x,eos), dim=0).to(torch.int64).to(device)
        # src [sent len, batch size]
        embedded = self.embedding(x) # [sent len, batch size, emb dim]
        outputs, hidden = self.rnn(embedded)  # [sent len, batch size, hidden dim]
        # outputs -> [sent len, batch size, hidden dim * n directions]
        # hidden -> [n layers * n directions, batch size, hidden dim]


        # initial decoder hidden is final hidden state of the forwards and
        # backwards encoder RNNs fed through a linear layer
        concated = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)

        hidden = torch.tanh(self.fc(concated))
        return outputs, hidden

In [24]:
class Attention(nn.Module):

    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        # enc_hid_dim multiply by 2 due to bidirectional
        self.fc1 = nn.Linear(enc_hid_dim * 2 + dec_hid_dim, dec_hid_dim)
        self.fc2 = nn.Linear(dec_hid_dim, 1, bias=False)

    def forward(self, encoder_outputs, hidden):
        src_len = encoder_outputs.shape[0]
        batch_size = encoder_outputs.shape[1]

        # repeat encoder hidden state src_len times [batch size, sent len, dec hid dim]
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        # reshape/permute the encoder output, so that the batch size comes first
        # [batch size, sent len, enc hid dim * 2], times 2 because of bidirectional
        outputs = encoder_outputs.permute(1, 0, 2)

        # the attention mechanism receives a concatenation of the hidden state
        # and the encoder output

        concat = torch.cat((hidden, outputs), dim=2)

        # fully connected layer and softmax layer to compute the attention weight
        # [batch size, sent len, dec hid dim]
        energy = torch.tanh(self.fc1(concat))

        # attention weight should be of [batch size, sent len]
        attention = self.fc2(energy).squeeze(dim=2)
        attention_weight = torch.softmax(attention, dim=1)
        return attention_weight

In [25]:
class Decoder(nn.Module):

    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, n_layers,
                 dropout, attention):
        super().__init__()
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.n_layers = n_layers
        self.dropout = dropout
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU(enc_hid_dim * 2 + emb_dim, dec_hid_dim, n_layers, dropout=dropout)
        self.linear = nn.Linear(dec_hid_dim, output_dim)

    def forward(self, trg, encoder_outputs, hidden):
        # trg [batch size]
        # outputs [src sen len, batch size, enc hid dim * 2], times 2 due to bidirectional
        # hidden [batch size, dec hid dim]

        # [batch size, 1, sent len]
        attention = self.attention(encoder_outputs, hidden).unsqueeze(1)

        # [batch size, sent len, enc hid dim * 2]
        outputs = encoder_outputs.permute(1, 0, 2)

        # [1, batch size, enc hid dim * 2]
        context = torch.bmm(attention, outputs).permute(1, 0, 2)

        # input sentence -> embedding
        # [1, batch size, emb dim]
        embedded = self.embedding(trg.unsqueeze(0))
        rnn_input = torch.cat((embedded, context), dim=2)

        outputs, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        prediction = self.linear(outputs.squeeze(0))
        return prediction, hidden.squeeze(0)


In [26]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src_batch, trg_batch, teacher_forcing_ratio=0.5):
        max_len, batch_size = trg_batch.shape
        trg_vocab_size = self.decoder.output_dim

        # tensor to store decoder's output
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        # encoder_outputs : all hidden states of the input sequence (forward and backward)
        # hidden : final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src_batch)

        trg = trg_batch[0]
        # tt = torch.transpose(trg_batch,1,0)[0]
        # tar_srt = [chr(it) for it in tt]
        # print("S2S", f"{''.join(tar_srt)}")
        # stri = [chr(trg[0])]
        for i in range(1, max_len):
            # stri.append(chr(trg[0]))
            prediction, hidden = self.decoder(trg, encoder_outputs, hidden)
            # print("S2S_pred", prediction.shape)
            outputs[i] = prediction

            if random.random() < teacher_forcing_ratio:
                trg = trg_batch[i]
            else:
                trg = prediction.argmax(1)
                # print("trg", trg)
        # print(f"pred: {''.join(stri)}")

        return outputs

In [27]:
attention = Attention(ENC_HID_DIM, DEC_HID_DIM)
encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, N_LAYERS, ENC_DROPOUT)
decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, N_LAYERS, DEC_DROPOUT, attention)
seq2seq = Seq2Seq(encoder, decoder, device).to(device)
seq2seq

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Seq2Seq(
  (encoder): Encoder(
    (conv): Conv2d(1, 1, kernel_size=(1, 4), stride=(1, 1))
    (pool): MaxPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0, dilation=1, ceil_mode=False)
    (embedding): Embedding(1135, 128)
    (rnn): GRU(128, 256, dropout=0.5, bidirectional=True)
    (fc): Linear(in_features=512, out_features=256, bias=True)
  )
  (decoder): Decoder(
    (attention): Attention(
      (fc1): Linear(in_features=768, out_features=256, bias=True)
      (fc2): Linear(in_features=256, out_features=1, bias=False)
    )
    (embedding): Embedding(127, 128)
    (rnn): GRU(640, 256, dropout=0.5)
    (linear): Linear(in_features=256, out_features=127, bias=True)
  )
)

# Train

In [28]:
import time

In [29]:
optimizer = optim.Adam(seq2seq.parameters())

# ignore the padding index when calculating the loss
criterion = nn.CrossEntropyLoss(ignore_index=101).to(device)

In [30]:
def train(seq2seq, iterator, optimizer, criterion):
    seq2seq.train()

    epoch_acc  = 0
    epoch_loss = 0
    for batch in iterator:
        optimizer.zero_grad()

        brain = np.round(batch["brain"],2)[:,None,:,:].type(torch.float).to(device)
        tg = torch.transpose(batch["target"],1,0).to(device)


        outputs =seq2seq(brain, tg)

        # the loss function only works on 2d inputs
        # and 1d targets we need to flatten each of them
        outputs_flatten = outputs[1:].view(-1, outputs.shape[-1]).to(device)
        trg_flatten = tg[1:].contiguous().view(-1).to(device)

        logits = torch.argmax(outputs, dim=2).type(torch.float).to(device)
        loss = criterion(outputs_flatten, trg_flatten)

        loss.backward()
        optimizer.step()
        epoch_acc += torch.mean((logits == tg).type(torch.float))
        epoch_loss += loss.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [31]:
def evaluate(seq2seq, iterator, criterion):
    seq2seq.eval()

    epoch_acc  = 0
    epoch_loss = 0
    with torch.no_grad():
        for batch in iterator:
            # turn off teacher forcing
            brain = np.round(batch["brain"],2)[:,None,:,:].type(torch.float).to(device)
            tg = torch.transpose(batch["target"],1,0).to(device)


            outputs =seq2seq(brain, tg, teacher_forcing_ratio=0)

            # trg = [trg sent len, batch size]
            # output = [trg sent len, batch size, output dim]
            outputs_flatten = outputs[1:].view(-1, outputs.shape[-1]).to(device)
            trg_flatten = tg[1:].contiguous().view(-1).to(device)

            logits = torch.argmax(outputs, dim=2).type(torch.float).to(device)
            loss = criterion(outputs_flatten, trg_flatten)

            epoch_acc += torch.mean((logits == tg).type(torch.float))
            epoch_loss += loss.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator),

In [52]:
N_EPOCHS = 30
best_valid_loss = float('inf')
train_loses =[]
test_loses =[]

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc = train(seq2seq, train_dataloader, optimizer, criterion)
    valid_loss, val_acc = evaluate(seq2seq, test_dataloader, criterion)
    end_time = time.time()

    train_loses.append(train_loss)
    test_loses.append(valid_loss)

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(seq2seq.state_dict(), 'tut2-model.pt')

    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f}')

Epoch: 01
	Train Loss: 0.930
	 Val. Loss: 0.929
Epoch: 02
	Train Loss: 0.934
	 Val. Loss: 0.951
Epoch: 03
	Train Loss: 0.986
	 Val. Loss: 0.997
Epoch: 04
	Train Loss: 0.931
	 Val. Loss: 0.914
Epoch: 05
	Train Loss: 0.918
	 Val. Loss: 0.939
Epoch: 06
	Train Loss: 0.834
	 Val. Loss: 0.999
Epoch: 07
	Train Loss: 0.875
	 Val. Loss: 0.920
Epoch: 08
	Train Loss: 0.845
	 Val. Loss: 0.947
Epoch: 09
	Train Loss: 0.856
	 Val. Loss: 0.918
Epoch: 10
	Train Loss: 0.839
	 Val. Loss: 0.969
Epoch: 11
	Train Loss: 0.854
	 Val. Loss: 0.991
Epoch: 12
	Train Loss: 0.868
	 Val. Loss: 0.927
Epoch: 13
	Train Loss: 0.896
	 Val. Loss: 0.974
Epoch: 14
	Train Loss: 0.608
	 Val. Loss: 0.850
Epoch: 15
	Train Loss: 0.629
	 Val. Loss: 0.829
Epoch: 16
	Train Loss: 0.653
	 Val. Loss: 0.857
Epoch: 17
	Train Loss: 0.687
	 Val. Loss: 0.834
Epoch: 18
	Train Loss: 0.668
	 Val. Loss: 0.845
Epoch: 19
	Train Loss: 0.691
	 Val. Loss: 0.825
Epoch: 20
	Train Loss: 0.662
	 Val. Loss: 0.819
Epoch: 21
	Train Loss: 0.604
	 Val. Loss

In [54]:
for j in train_dataloader:
    break

brain = np.round(j["brain"],2)[:,None,:,:].type(torch.float).to(device)
tg = torch.transpose(j["target"],1,0).to(device)

seq2seq.eval()
with torch.no_grad():
    outputs = seq2seq(brain, tg, teacher_forcing_ratio=0)

outputs.shape

torch.Size([128, 10, 127])


In [55]:
def calculate_wer(decoded_sentences, true_sentences):
    total_word_errors = 0
    total_words = 0

    for decoded_sent, true_sent in zip(decoded_sentences, true_sentences):
        decoded_words = decoded_sent.split(" ")
        true_words = true_sent.split(" ")
        word_errors = editdistance.eval(decoded_words, true_words)
        total_word_errors += word_errors
        total_words += len(true_words)

    wer = total_word_errors / total_words
    return wer

In [56]:
wer = calculate_wer(outputs.argmax(2),tg)
print(np.round(wer,4))

15.2584


In [57]:
!pip freeze > requirements.txt